In [2]:
import serial
import time
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from collections import Counter

# import matplotlib as mpl


port = '/dev/tty.usbmodem144050001' #131357501
ser = serial.Serial(port, 115200)

2024-09-11 22:37:37.069355: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 22:37:37.073545: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 22:37:37.087172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 22:37:37.109887: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 22:37:37.116536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 22:37:37.132637: I tensorflow/core/platform/cpu_feature_gu

SerialException: [Errno 2] could not open port /dev/tty.usbmodem144050001: [Errno 2] No such file or directory: '/dev/tty.usbmodem144050001'

# microGesture


In [1]:
def analyze_txt_and_predict_force(model_file_path, write_file_name, write_file_path_root, frame_interval, window_size, force_threshold, result_map):
    if frame_interval <= window_size:
        return analyze_txt_and_predict_force_overlap(model_file_path, write_file_name, write_file_path_root, frame_interval, window_size, force_threshold, result_map)
    else:
        return analyze_txt_and_predict_force_nonOverlap(model_file_path, write_file_name, write_file_path_root, frame_interval, window_size, force_threshold, result_map)
def analyze_txt_and_predict_force_overlap(model_file_path, write_file_name, write_file_path_root, frame_interval, window_size, force_threshold, result_map):
    def normalize(raw_data):
        mean = np.mean(raw_data, axis=0)
        std = np.std(raw_data, axis=0)
        normalized_data = (raw_data - mean) / std
        return normalized_data
    
    model = load_model(model_file_path)

    # Temporary lists to hold data before packing into the main lists (as a queue)
    temp_real_queue = []
    temp_imaginary_queue = []
    temp_force_queue = []
    temp_gesture_queue = []

    y_test = []
    y_pred = []
    x_test_list = []
    x_test_list_preNormalize = []
    if not os.path.exists(write_file_path_root):
        os.makedirs(write_file_path_root)
    write_file_path = os.path.join(write_file_path_root, write_file_name)
    with open(write_file_path, 'w') as f:
        frame_count = 0  # To track the total number of frames processed
        extracting_frames = False  # Flag to indicate if we are in an extraction block
        lines_in_frame = 0  # To count lines within a frame block
        predict_size = 0
        while True:
        # with open(read_file_path, 'r') as file:
            # for line in file:
                line = ser.readline().decode()
                line = line.strip()
                line_part = line.split()
                # Detect the start of a new frame
                if line_part[0] == "frame":
                    frame_count += 1
                    lines_in_frame = 0  # Reset line count for the new frame
                    extracting_frames = True  # Always extracting frames

                # If we're extracting frames, process the next 3 lines
                if extracting_frames:
                    temp_gesture_queue.append(line_part[1])
                    if lines_in_frame == 1:  # First data line (real values)
                        temp_real_queue.append([float(num) for num in line.split()])
                    elif lines_in_frame == 2:  # Second data line (imaginary values)
                        temp_imaginary_queue.append([float(num) for num in line.split()])
                    elif lines_in_frame == 3:  # Third data line (force values)
                        force_value = float(line.split()[0])  # Extract the first value (force)
                        temp_force_queue.append(force_value)

                        # If the queue length reaches frames_to_extract, perform prediction
                        if len(temp_real_queue) == window_size:
                            # Convert queue data to numpy arrays
                            np_real_list = np.array(temp_real_queue)
                            np_imaginary_list = np.array(temp_imaginary_queue)
                            np_force_list = np.array(temp_force_queue)
                            np_force_mean_value = np.mean(np_force_list)

                            gesture_word_counts = Counter(temp_gesture_queue)
                            y_test_value = gesture_word_counts.most_common(1)[0][0]


                            if np_force_mean_value < force_threshold:
                                np_force_mean_value = 0
                            # Combine real and imaginary parts
                            combined_12 = np.concatenate((np_real_list, np_imaginary_list), axis=1)
                            x_test_list_preNormalize.append(combined_12)
                            # combined_12 = normalize(combined_12)
                            x_test_list.append(combined_12)
                            X_test = np.expand_dims(combined_12, axis=0)
                            # Perform prediction
                            y_pred_value = model.predict(X_test)
                            y_pred_labels = np.argmax(y_pred_value, axis=1)
                            predict_size += 1
                            y_pred.append(result_map[y_pred_labels[0]])
                            y_test.append(np_force_mean_value)
                            # Write results to file
                            f.write(f"Predict {predict_size}: \n")
                            f.write(f"force mean: {np_force_mean_value}\n")
                            f.write(f"ground truth frame type: {y_test_value}\n")
                            f.write(f"predict frame type: {result_map[y_pred_labels[0]]}\n")

                            # Remove the oldest frame to allow overlap
                            for delete in range(frame_interval):
                                temp_real_queue.pop(0)
                                temp_imaginary_queue.pop(0)
                                temp_force_queue.pop(0)

                # Count lines within a frame block
                if extracting_frames:
                    lines_in_frame += 1
        ser.close()
    f.close()
    y_pred = np.array(y_pred)
    y_test = np.array(y_test)
    y_pred_write_file_path = os.path.join(write_file_path_root, "y_pred")
    np.save(y_pred_write_file_path, y_pred)
    y_test_write_file_path = os.path.join(write_file_path_root, "y_test")
    np.save(y_test_write_file_path, y_test)
    return y_pred, y_test, np.array(x_test_list), np.array(x_test_list_preNormalize)

def analyze_txt_and_predict_force_nonOverlap(model_file_path, read_file_path, write_file_name, write_file_path_root, frame_interval, window_size, force_threshold, result_map):
    def normalize(raw_data):
        mean = np.mean(raw_data, axis=0)
        std = np.std(raw_data, axis=0)
        normalized_data = (raw_data - mean) / std
        return normalized_data
    model = load_model(model_file_path)

    y_test = []
    y_pred = []
    x_test_list = []
    x_test_list_preNormalize = []
    if not os.path.exists(write_file_path_root):
        os.makedirs(write_file_path_root)
    write_file_path = os.path.join(write_file_path_root, write_file_name)
    with open(write_file_path, 'w') as f:
        frame_count = 0  # To track the total number of frames processed
        extracting_frames = False  # Flag to indicate if we are in an extraction block
        lines_in_frame = 0  # To count lines within a frame block
        predict_size = 0
        while True:
        # with open(read_file_path, 'r') as file:
            # for line in file:
                line = ser.readline().decode()
                line = line.strip()
                line_part = line.split()
                # Detect the start of a new frame
                if line_part[0] == "frame":
                    frame_count += 1
                    lines_in_frame = 0  # Reset line count for the new frame

                    # Check if it's time to start extracting frames
                    if frame_count % frame_interval == 0:
                        extracting_frames = True
                        frames_extracted = 0  # Reset the number of extracted frames for this block
                        temp_real_list = []  # Clear temp lists for this extraction block
                        temp_imaginary_list = []
                        temp_force_list = []
                        temp_gesture_list = []

                # If we're extracting frames, process the next 3 lines
                if extracting_frames:
                    temp_gesture_list.append(line_part[1])
                    if lines_in_frame == 1:  # First data line (real values)
                        temp_real_list.append([float(num) for num in line.split()])
                    elif lines_in_frame == 2:  # Second data line (imaginary values)
                        temp_imaginary_list.append([float(num) for num in line.split()])
                    elif lines_in_frame == 3:  # Third data line (force values)
                        force_value = float(line.split()[0])  # Extract the first value (force)
                        temp_force_list.append(force_value)
                        frames_extracted += 1  # Increment frames extracted count


                        # If we have extracted enough frames (frames_to_extract), store them
                        if frames_extracted == window_size:
                            # Convert queue data to numpy arrays
                            np_real_list = np.array(temp_real_list)
                            np_imaginary_list = np.array(temp_imaginary_list)
                            np_force_list = np.array(temp_force_list)
                            np_force_mean_value = np.mean(np_force_list)

                            gesture_word_counts = Counter(temp_gesture_list)
                            y_test_value = gesture_word_counts.most_common(1)[0][0]

                            if np_force_mean_value < force_threshold:
                                np_force_mean_value = 0
                            # Combine real and imaginary parts
                            combined_12 = np.concatenate((np_real_list, np_imaginary_list), axis=1)
                            x_test_list_preNormalize.append(combined_12)
                            # combined_12 = normalize(combined_12)
                            x_test_list.append(combined_12)
                            X_test = np.expand_dims(combined_12, axis=0)
                            # Perform prediction
                            y_pred_value = model.predict(X_test)
                            y_pred_labels = np.argmax(y_pred_value, axis=1)
                            predict_size += 1

                            y_pred.append(result_map[y_pred_labels[0]])
                            y_test.append(np_force_mean_value)
                            # Write results to file
                            f.write(f"Predict {predict_size}: \n")
                            f.write(f"force mean: {np_force_mean_value}\n")
                            f.write(f"ground truth frame type: {y_test_value}\n")
                            f.write(f"predict frame type: {result_map[y_pred_labels[0]]}\n")

                            extracting_frames = False  # Stop extracting for this interval

                # Count lines within a frame block
                if extracting_frames:
                    lines_in_frame += 1
        ser.close()
    f.close()
    y_pred = np.array(y_pred)
    y_test = np.array(y_test)
    y_pred_write_file_path = os.path.join(write_file_path_root, "y_pred")
    np.save(y_pred_write_file_path, y_pred)
    y_test_write_file_path = os.path.join(write_file_path_root, "y_test")
    np.save(y_test_write_file_path, y_test)
    return y_pred, y_test, np.array(x_test_list), np.array(x_test_list_preNormalize)

model_file_path = "/home/ubuntu/paper_arm_model/best_model/best_gesture_model.h5"
write_file_name = "microGesture_predict_result_and_ground_truth.txt"
write_file_path_root = "/home/ubuntu/paper_arm_model/best_model_result/test2"
frame_interval = 2  # Number of frames to skip between extractions
window_size = 60  # Number of frames to extract after the interval
force_threshold = 0.5
result_map = ['id', 'ipr', 'ips', 'mpr' ,'mps', 'sl', 'sr']

y_pred, y_test, x_test_list, x_test_list_preNormalize = analyze_txt_and_predict_force(model_file_path, write_file_name, write_file_path_root, frame_interval, window_size, force_threshold,result_map)


NameError: name 'load_model' is not defined